Crawler que acessa o todos os imagesets indicados e cria/baixa um tag específica

In [87]:
import sys
import getpass
import shutil
import os
from lxml import html
try:
    import requests
except ImportError:
    print("Python3 requests is not installed. Please use e.g. pip3 install requests")
    sys.exit()

imageset = [258, 157, 160, 258, 196, 197, 168, 166, 2, 262, 261, 260, 20, 19, 21, 149, 233, 159, 81, 34, 155, 154, 156, 153, 35, 36, 33, 25, 5, 7, 12, 13, 14, 15, 16, 17, 18, 29, 30, 31, 32, 33]

print('N datasets: {}'.format(len(imageset)))

s = requests.Session()

BaseUrl = "https://imagetagger.bit-bots.de" + "/"
user = "pequi"
password = "pequimecanico"
filename="data"
if not os.path.exists(os.getcwd() + '/' +filename):
    os.makedirs(os.getcwd()+'/'+filename)

loginpage = s.get(BaseUrl)

csrftoken = loginpage.cookies['csrftoken']
csrfmiddlewaretoken = csrftoken

data = {'username': user,
        'password': password,
        'csrfmiddlewaretoken': csrfmiddlewaretoken}
loggedinpage = s.post(
    '{}user/login/'.format(BaseUrl),
    data=data,
    allow_redirects=False,
    headers={'referer': BaseUrl})

try:
	sessionid  = loggedinpage.cookies['sessionid']
except:
	print('Login failed')

N datasets: 42


In [101]:
TAGS_PATH = os.path.abspath('tags')
TAG_ID = 33 #No select do site
TAG_NAME = 'base' #Nome da tag

def download_tag(current_imageset, tag_number, tag_name):
    current_imageset = str(current_imageset)
    if(os.path.isfile(os.path.join(TAGS_PATH, '{}.txt'.format(current_imageset)))):
        return
    page = s.get("{}images/imageset/{}/".format(BaseUrl,current_imageset))
    if(page.status_code != 200):
        return
    page_html = html.document_fromstring(page.text)
    tag_notfound = True
    #Se quiser sempre gerar uma tag nova mesmo que ja exista remova este 'for-loop' abaixo.
    for h3 in page_html.cssselect('h3'):
        if(h3.text_content() == 'Download exports:'):
            for a in h3.getparent().cssselect('a'):
                if(a.text_content().split(':')[0] == tag_name):
                    download_link = BaseUrl[:-1] + a.get('href')
                    tag_notfound = False
                    tag = s.get(download_link, allow_redirects=True)
                    open(os.path.join(TAGS_PATH, '{}.txt'.format(current_imageset)), 'wb').write(tag.content)
                    break
    if(tag_notfound):
        csrfmiddlewaretoken = page_html.cssselect('input[name=csrfmiddlewaretoken]')[0].get('value')
        data = {'export_format': str(tag_number),
                'export': 'Export',
                'csrfmiddlewaretoken': csrfmiddlewaretoken}
        s.post("{}annotations/export/{}/".format(BaseUrl,current_imageset)
               ,data = data
               ,allow_redirects=False
               ,headers={'referer': BaseUrl})
        download_tag(current_imageset, tag_number, tag_name)

for dataset in os.listdir('data'):
    download_tag(dataset, TAG_ID, TAG_NAME)